In [1]:
import pandas as pd
import glob as gb
import os

files = gb.glob('./data/Ward_Age/*')
dataname = '町丁目名年齢年代別人口分布'

d=os.getcwd()
prefix = d + '/../data/Ward_Age/'


# c
#g_code = pd.read_csv(prefix+'東京都人口世帯数_estat.txt', encoding='SHIFT-JIS', dtype={'KEY_CODE':str})
#g_code[g_code['HYOSYO']>3]
#g_code=g_code[['KEY_CODE','NAME','CITYNAME']]
wds = ['市区町村コード','市区町村名','大字町丁目コード','大字町丁目名']
c_code = pd.read_csv(prefix + '13_2020.csv', encoding='utf-8', dtype={'大字町丁目コード':str})[wds]

## 1. 大田区

In [2]:
ohta_file = prefix + '大田区2022年1月_rev.xlsx'

ohta_data = pd.read_excel(ohta_file).set_index('年齢')

ohta_finedata = pd.DataFrame(ohta_data.stack()).reset_index().set_axis(['年齢','町丁目名','人口'],axis=1)

ohta_finedata = ohta_finedata[['町丁目名','年齢','人口']]

ohta_finedata['町丁目名'] = ohta_finedata['町丁目名'].str.replace('　','')

ohta_finedata = pd.merge(ohta_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

ohta_finedata['区']='大田区'

In [3]:
#ohta_finedata.to_excel(prefix + dataname + '（大田）.xlsx', index=False)

bin_band =  list(range(0,131,5))
ew=pd.cut(ohta_finedata['年齢'], bin_band, right=False)
#pd.DataFrame(pd.DataFrame(ew).value_counts()).sort_values('年齢')

## 2. 品川

In [4]:
male=list(range(0,241,2))
female=list(range(1,241,2))

shina_file = prefix + '品川区2023年1月_rev.xls'
shina_data = pd.read_excel(shina_file, dtype={'行政コード':int})

shina_m = shina_data.loc[:,['行政コード','丁目名']+male].set_index(['行政コード','丁目名']).stack()
shina_f = shina_data.loc[:,['行政コード','丁目名']+female].set_index(['行政コード','丁目名']).stack()

shina_m_fine= pd.DataFrame(shina_m).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_m_fine['年齢']=shina_m_fine['age']//2

shina_f_fine= pd.DataFrame(shina_f).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_f_fine['年齢']=(shina_f_fine['age'])//2

scols=['丁目名','年齢','人口']
mer=['丁目名','年齢']

shina_m_fine=shina_m_fine.drop(columns=['age'])[scols]
shina_f_fine=shina_f_fine.drop(columns=['age'])[scols]


shina_finedata = pd.merge(shina_m_fine, shina_f_fine, on=mer)

#for t,i in shina_finedata.iterrows():
#    shina_finedata.loc[i,'丁目名'] = shina_finedata.loc[i, '丁目名'].replace('\u3000','')
#    print(shina_finedata.loc[i, '丁目名'])

shina_finedata['人口'] = shina_finedata['人口_x']+shina_finedata['人口_y']
shina_finedata = shina_finedata.drop(columns=['人口_x','人口_y']).reset_index(drop=True)
shina_finedata = shina_finedata.rename(columns={'丁目名':'町丁目名'})
shina_finedata = pd.merge(shina_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

shina_finedata['区']='品川区'

In [5]:
shina_finedata

,町丁目名,年齢,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,区
0,北品川１丁目,0,20,13109,品川区,131090019001,北品川１丁目,品川区
1,北品川１丁目,1,17,13109,品川区,131090019001,北品川１丁目,品川区
2,北品川１丁目,2,21,13109,品川区,131090019001,北品川１丁目,品川区
3,北品川１丁目,3,18,13109,品川区,131090019001,北品川１丁目,品川区
4,北品川１丁目,4,14,13109,品川区,131090019001,北品川１丁目,品川区
...,...,...,...,...,...,...,...,...
16195,東八潮,115,0,13109,品川区,131090005000,東八潮,品川区
16196,東八潮,116,0,13109,品川区,131090005000,東八潮,品川区
16197,東八潮,117,0,13109,品川区,131090005000,東八潮,品川区
16198,東八潮,118,0,13109,品川区,131090005000,東八潮,品川区


In [6]:
#shina_finedata.to_excel(prefix + dataname + '（品川）.xlsx', index=False)

## 3. 目黒

In [7]:
meg_file = prefix +'目黒区2022年12月_rev.xlsx'
meg_data = pd.read_excel(meg_file)
#meg_data

top = meg_data[meg_data['a']=='町丁別年齢別人口表'].reset_index()['index']
end = meg_data[meg_data['a']=='104以上'].reset_index()['index']
#print(list(top))
#print(list(end))

meg_tmp = pd.DataFrame()

#meg_data['町丁目名']=''

for k in top:
    chomei = meg_data.iloc[k+1,0].replace('　','')

    for j in list(range(k+4,k+129)):
        meg_data.loc[j,'町丁目名'] = chomei

    chodf = meg_data.iloc[(k+4):(k+129),:]
    
    chodf = chodf[~(chodf['a'].str.contains('～',na=False))]
    chodf = chodf[~(chodf['a'].str.contains('以上',na=False))]
    
    chodf = chodf.rename(columns={'a':'年齢', 'b':'人口'})
    chodf['年齢'] = chodf['年齢'].astype(int)
    
    meg_tmp = pd.concat([meg_tmp, chodf])

meg_finedata = meg_tmp[['町丁目名','年齢','人口']]
meg_finedata = pd.merge(meg_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

meg_finedata['区']='目黒区'

In [8]:
#meg_finedata.to_excel(prefix + dataname + '（目黒）.xlsx', index=False)

## 4. 港区

In [9]:
import tabula

mina_file = prefix + '港区2022年4月.pdf'

mina_data = tabula.read_pdf(mina_file,pages="1-117",stream=True)

In [10]:
textfile = open(prefix+'港区町丁目名.txt','r')
output_text = textfile.read()
textfile.close()

import re

pattern = '(.+丁目)|(麻布.+町)|(麻布狸穴町)$'

minacho = re.findall(pattern, output_text)

from itertools import chain
def flatten(nested_list):
    return list(chain.from_iterable(nested_list))

#flatten(list(map(lambda x: list(x), minacho)))

chomeilist = list(map((lambda y: ''.join(y)), list(map(lambda x: list(x), minacho))))

In [11]:
mina_finedata = pd.DataFrame()

for i in range(0,116):
    if i != 39:
        k = mina_data[i].iloc[:21,0:3].set_axis(['世代','a','b'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k = k.fillna(0)
        k = k.astype({'a':str, 'b':str})
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)
        k['b'] = k['b'].str.replace(',','')
        k['b'] = k['b'].str.replace(r'0\.0','0', regex=True)
    else:
        k = mina_data[i].iloc[:21,0:2].set_axis(['世代','a'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)        
        k['b'] = '0'
        
    mina_finedata = pd.concat([mina_finedata, k])

mina_finedata = mina_finedata.astype({'a':int, 'b':int}).reset_index(drop=True) 

mina_finedata['人口'] = mina_finedata['a'] + mina_finedata['b']
#

for r in mina_finedata.itertuples():
    if not(((r.a == 0) and (r.b==r.人口)) or ((r.b==0) and (r.a==r.人口))):
        print(r.町丁目名, ' ', r.a, r.b)

mina_finedata = mina_finedata.drop(columns=['a','b'])[['町丁目名','世代','人口']]
mina_finedata = pd.merge(mina_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

mina_finedata['区']='港区'

In [12]:
mina_finedata

,町丁目名,世代,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,区
0,赤坂１丁目,0~4歳,35,13103,港区,131030008001,赤坂１丁目,港区
1,赤坂１丁目,5~9歳,37,13103,港区,131030008001,赤坂１丁目,港区
2,赤坂１丁目,10~14歳,25,13103,港区,131030008001,赤坂１丁目,港区
3,赤坂１丁目,15~19歳,22,13103,港区,131030008001,赤坂１丁目,港区
4,赤坂１丁目,20~24歳,20,13103,港区,131030008001,赤坂１丁目,港区
...,...,...,...,...,...,...,...,...
2515,台場１丁目,80~84歳,89,13103,港区,131030010001,台場１丁目,港区
2516,台場１丁目,85~89歳,50,13103,港区,131030010001,台場１丁目,港区
2517,台場１丁目,90~94歳,19,13103,港区,131030010001,台場１丁目,港区
2518,台場１丁目,95~99歳,7,13103,港区,131030010001,台場１丁目,港区


In [13]:
#mina_finedata.to_excel(prefix + dataname + '（港）.xlsx', index=False)

## 5. 渋谷

In [14]:
shib_data = pd.read_excel(prefix + '渋谷区2022年1月_rev.xlsx')

shib_finedata = shib_data[['町丁名','年齢','人口']].rename(columns={'町丁名':'町丁目名'})

shib_finedata = pd.merge(shib_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

shib_finedata['区']='渋谷区'

In [15]:
#shib_finedata.to_excel(prefix + dataname + '（渋谷）.xlsx', index=False)

### 6. 台東

### データ統合

In [16]:
finedata = pd.concat([ohta_finedata,
                      meg_finedata,
                      shib_finedata,
                      shina_finedata
                      ]).reset_index(drop=True)

finedata['世代'] = ''

# fiveband = list(mina_finedata.loc[:20,:]['世代'])
#print(fiveband)

# ['0~4歳', '5~9歳', '10~14歳', '15~19歳', '20~24歳', '25~29歳', '30~34歳',
# '35~39歳', '40~44歳', '45~49歳', '50~54歳', '55~59歳', '60~64歳', '65~69歳',
# '70~74歳', '75~79歳', '80~84歳', '85~89歳', '90~94歳', '95~99歳', '100~104歳'
# '105~109歳', '110~114歳', '115~119歳', '120歳以上']

# 0~130まで5づつでビンを作成
bin_band =  list(range(0,131,5))
# binのラベル
fiveband = ['0~4歳', '5~9歳', '10~14歳', '15~19歳', '20~24歳', '25~29歳', '30~34歳',
 '35~39歳', '40~44歳', '45~49歳', '50~54歳', '55~59歳', '60~64歳', '65~69歳',
 '70~74歳', '75~79歳', '80~84歳', '85~89歳', '90~94歳', '95~99歳', '100~104歳',
 '105~109歳', '110~114歳', '115~119歳', '120~124歳', '125~129歳']#, '130歳以上']

fiveband_df = pd.DataFrame(
            {
                'gord' : list(range(0,26)),
                '世代': fiveband
            }
)


finedatas = finedata[finedata['人口']>0]

finedata['世代']=pd.cut(finedata['年齢'], bin_band, right=False, labels=fiveband)

finedata = pd.merge(finedata, fiveband_df, on='世代', how='left')


In [17]:
finedata

,町丁目名,年齢,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,区,世代,gord
0,大森東１丁目,0,24,13111,大田区,131110039001,大森東１丁目,大田区,0~4歳,0
1,大森東１丁目,1,28,13111,大田区,131110039001,大森東１丁目,大田区,0~4歳,0
2,大森東１丁目,2,36,13111,大田区,131110039001,大森東１丁目,大田区,0~4歳,0
3,大森東１丁目,3,36,13111,大田区,131110039001,大森東１丁目,大田区,0~4歳,0
4,大森東１丁目,4,31,13111,大田区,131110039001,大森東１丁目,大田区,0~4歳,0
...,...,...,...,...,...,...,...,...,...,...
67584,東八潮,115,0,13109,品川区,131090005000,東八潮,品川区,115~119歳,23
67585,東八潮,116,0,13109,品川区,131090005000,東八潮,品川区,115~119歳,23
67586,東八潮,117,0,13109,品川区,131090005000,東八潮,品川区,115~119歳,23
67587,東八潮,118,0,13109,品川区,131090005000,東八潮,品川区,115~119歳,23


In [18]:
fd_grp = ['区','町丁目名','大字町丁目コード','世代','gord']
srt = ['大字町丁目コード','gord']

final = pd.DataFrame(finedata.groupby(fd_grp)['人口'].agg(sum)).sort_values(srt).reset_index()

mina_final = pd.merge(mina_finedata, fiveband_df, on='世代')[fd_grp+['人口']]

final = pd.concat([final, mina_final])

In [19]:
final

,区,町丁目名,大字町丁目コード,世代,gord,人口
0,目黒区,三田１丁目,131030019001,0~4歳,0,76
1,目黒区,三田１丁目,131030019001,5~9歳,1,60
2,目黒区,三田１丁目,131030019001,10~14歳,2,47
3,目黒区,三田１丁目,131030019001,15~19歳,3,57
4,目黒区,三田１丁目,131030019001,20~24歳,4,90
...,...,...,...,...,...,...
2395,港区,六本木４丁目,131030028004,95~99歳,19,3
2396,港区,六本木５丁目,131030028005,95~99歳,19,6
2397,港区,六本木６丁目,131030028006,95~99歳,19,9
2398,港区,六本木７丁目,131030028007,95~99歳,19,11


In [20]:
final = final.astype({'大字町丁目コード':str})

In [21]:
d2k = {
    '１丁':'一丁',
    '２丁':'二丁',
    '３丁':'三丁',
    '４丁':'四丁',
    '５丁':'五丁',
    '６丁':'六丁',
    '７丁':'七丁',
    '８丁':'八丁',
    '９丁':'九丁'
}

for i in d2k.keys():
    final['町丁目名']=final['町丁目名'].str.replace(i,d2k[i])


In [33]:
final.to_csv(prefix+'final_data.csv', encoding='utf-8', index=False)

### ヒートマップ

In [23]:
d

'/Users/mori/Documents/Devel/Tokyo_Ward_Assembly_Election/jupyter'

In [24]:
import geopandas as gpd
import folium
import matplotlib
import mapclassify
import numpy as np

geodir = d + '/../geoshape/'

ward=['港区','目黒区','大田区','品川区','渋谷区']
w=ward[0]
f=fiveband[10]
print(w,f)



gdf = gpd.read_file(geodir + w + '.geojson')
gdf = gdf[gdf['HCODE']==8101]
kgdf = pd.merge(final, gdf, left_on=['区','町丁目名'], right_on=['CITY_NAME','S_NAME'], how='left')
kgdf = kgdf[kgdf['区']==w]
kgdf = kgdf[kgdf['世代']==f]
tmp_kgdf = gpd.GeoDataFrame(kgdf)

tmp_kgdf.explore(column=tmp_kgdf['人口'],cmap='Reds',tooltip=['町丁目名','人口'],tiles='CartoDB positron')


港区 50~54歳


In [25]:
gdf[['KEYCODE2','KEYCODE1','KIHON1','KIHON2','KCODE1','PREF','CITY','S_AREA','S_NAME']].head()

,KEYCODE2,KEYCODE1,KIHON1,KIHON2,KCODE1,PREF,CITY,S_AREA,S_NAME
0,103001001,103001001,0010,01,0010-01,13,103,001001,芝一丁目
1,103001002,103001002,0010,02,0010-02,13,103,001002,芝二丁目
2,103001003,103001003,0010,03,0010-03,13,103,001003,芝三丁目
3,103001004,103001004,0010,04,0010-04,13,103,001004,芝四丁目
4,103001005,103001005,0010,05,0010-05,13,103,001005,芝五丁目


In [26]:
#c_code[c_code['大字町丁目コード']=='131030001001']
#c_code.head()

gdf['S_NAME']=gdf['S_NAME'].str.replace('一丁','１丁').str.replace('二丁','２丁')

In [27]:
gdf['おお']=''

In [ ]:
gdf['S_NAME']

In [ ]:
#gdf[(gdf['CITY_NAME']=='港区') and (gdf['S_NAME']=='芝一丁目')]
gdf[(gdf['S_NAME']=='芝一丁目')]

In [ ]:
final[(final['町丁目名']=='芝１丁目')]